In [1]:
import json
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","ENVIRONMENT_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
paramDict = {}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)

w = Workflow(env, secret_store_cfg, None, global_vars=globals())




<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Notify about unused keypairs using unSkript legos.</b>
</div>

<br>

<center><h2>Notify about unused keypairs</h2></center>

# Steps Overview
    Find All the unused Keypairs from AWS instances
    Send message to Slack

Here we will use unSkript Filter AWS Unused KeyPairs Lego. This lego takes region as input. This inputs is used to findout the unused keypairs.

In [15]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_filter_unused_keypairs_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
def aws_filter_unused_keypairs(handle, region: str) -> List:
    """aws_filter_unused_keypairs Returns an array of KeyPair.

        :type region: object
        :param region: Object containing global params for the notebook.

        :rtype: Array of instances matching tags.
    """
    key_pairs_all = []
    used_key_pairs = []
    ec2Client = handle.client('ec2', region_name=region)

    # List the key pairs in the selected region
    key_pairs_all = list(map(lambda i: i['KeyName'], ec2Client.describe_key_pairs()['KeyPairs']))

    # Get group of EC2 instances.
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    for reservation in res:
        for keypair in reservation['Instances']:
            if 'KeyName'in keypair and keypair['KeyName'] not in used_key_pairs:
                used_key_pairs.append(keypair['KeyName'])

    # Compaire key_pairs_all and used_key_pairs list
    result = []
    for key in key_pairs_all:
        if key not in used_key_pairs:
            result.append(key)
    return result

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
task.configure(inputParamsJson='''{
    "region": "\\"us-west-2\\""
    }''')
task.configure(outputName="Unused_Keypair")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unused_keypairs, lego_printer=aws_filter_unused_keypairs_printer, hdl=hdl, args=args)

Here we will use unSkript Post Slack Message Lego. This lego takes channel: str and message: str as input. This inputs is used to post the message to the slack channel.

In [18]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "unSkript",
    "credential_type": "CONNECTOR_TYPE_SLACK",
    "credential_id": "b1adafe4-6593-484d-8fc1-e40799378e9a"
}''')
task.configure(inputParamsJson='''{
    "channel": "\\"dummy\\"",
    "message": "\\"Unused Keypair {}\\".format(Unused_Keypair)"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS to perform AWS Unused Keypairs and send a message on slack. To view the full platform capabilities of unSkript please visit https://unskript.com